## **Load *Monod*+meK-Means and Packages**

In [2]:
import sys
sys.path.insert(0, '/home/tchari/monod/src/')

In [3]:
import monod
from monod import preprocess, extract_data, cme_toolbox, inference, analysis, mminference

In [4]:
import pandas as pd
import numpy as np
import loompy as lp
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
import scipy.stats

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, ExtraTreesRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split, ShuffleSplit
from sklearn.metrics import accuracy_score, classification_report

import os

import sklearn
import scanpy as sc
import anndata

In [5]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## **Load Metadata and Select Genes for Inference Across Conditions**

In [6]:
#Load cell barcodes from original study

meta_path = '/home/tchari/metadata/'
data_path = '/home/tchari/counts/cancer_dt/'

In [8]:
orig_bcs = pd.read_csv(meta_path+'home/alex/lab/drugres/10x/D0D3FullSeq/Day3/outs/filtered_feature_bc_matrix/barcodes.tsv',header=None)
orig_bcs.head()


,0
0,AAACCCAAGCAGGCAT-1
1,AAACCCAAGCTGCCAC-1
2,AAACCCAAGGCCGCTT-1
3,AAACCCAAGTGACACG-1
4,AAACCCACACTTGGGC-1


In [10]:
#Get full count matrices and barcodes
ds = lp.connect('/home/tchari/counts/cancer_dt/loom/all_cancer_dt.loom')
U = ds.layers['unspliced'][:].T
S = ds.layers['spliced'][:].T
bars = ds.ca['barcode']
g_names = ds.ra['gene_name']
ds.close()

In [15]:
len(set(orig_bcs[0]).intersection(bars))/len(set(orig_bcs[0]))

0.971630015108276

In [1]:
#Make anndata with conditions and get most variable

adata = anndata.AnnData(S)
adata.obs_names = [f"Cell_{i:d}" for i in range(adata.n_obs)]
adata.var_names = g_names

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
#adata.var.highly_variable

In [ ]:
#Literature genes?
#Supp Fig 9

lit_genes = ['CDH2','COL1A1','CALD1','TGFB2', 'INHBA','INHBB','SERPINE1',
            'LEF1','TEAD1','SRF','FOXO4','FOXF2','FOXA1','ATF','E4F1','MAZ',
            'AP1','SOX9','E2F1','NFY','YY1','TGFB2','WNT5A','CTNNB1','DKK1','CDH2',
            'COL1A1','CALD1','SCD','FASN','FDFT1','LDLR','SQLE','HMGCS1','FADS2',
            'TNFRSF12A','FDPS','IDI1','ANXA5','ACAT2','FABP5','FAM129A','MVD',
            'TRIB3','ACSS2','DHCR7','ALCAM','SREBF2','HSD17B7','CBS',
            'STX5','JAG1','ERRFI1','LSS','ATF3','HMGCR','MAL2','CHKA',
            'MYL9','TAGLN','IGFBP3','TPM1','FSTL1','CALD1','FN1','GADD45A',
            'DAB2','GLIPR1','IL32','INHBA','CYR61','THBS1','MFAP5','SPARC',
            'SAT1','TFPI2','MGP','DKK1','FSTL3','WNT5A','SERPINH1','FERMT2',
            'GPC1','CTGF','TPM4','PDLIM4','ITGB1','SERPINE2','SDC4','CD59',
            'NOTCH2','COL5A1','LAMC1','PLAUR','BDNF','IGFBP3','FN1','CD24',
            'GADD45A','TGFB2','TIMP2','PSAP','CYR61','EDN1','INHBA','THBS1',
            'DUSP1','PINK1','GAS6','GSTP1','PDGFB','CTGF','TPD52L1','CAV2','CAV1',
            'INHBB','IGBP1','DAG1','PTPLAD1','ITGB1','MID1','LYN','PEA15',
            'RIPK2','RNF149','CDC42','PTPN11','EZR','TRIB1','MUC20','KRAS','RRAS',
            'GADD45A','CYR61','EDN1','SAT1','DUSP1','FOS','PNRC1','JUNB','TIPARP',
            'OLR1','F3','IER2','CCND1','CCNL1','NAMPT','TUBB2A','TRIB1','SDC4','BTG3',
            'RIPK2','PTGS2','TGIF1','PLAUR','ID3','SOX4','CYP1B1','KLHL24',
            'GSN','AHNAK2','GRN','HIST1H2AC','HIST1H2BD','HIST1H1C','HEG1','FTH1',
            'HCFC1R1','GLUL','CTSH','BCAM','CALCOCO1','MUC1','ANXA4','EPHX1','YPEL5']

## **Set up Files for Inference and Select Cell Barcodes**

In [902]:
ks = [5,10] #K options for meK-Means
num_ks = len(ks) #Number of Ks to try

In [903]:
#Set  up output file names for inference run results

#Make names for each of the K runs
clus_names = ['k'+str(run_k) for run_k in ks]

#Output file names
dataset_names = ['meK_norman_'+y  for y in clus_names]
print('Runs: ',dataset_names)

Runs:  ['meK_norman_k3', 'meK_norman_k5']


In [904]:
#Set reference data filepaths
transcriptome_filepath = '/home/tchari/perturbCME/notebooks/gg_200525_genome_polyA_cum_3'

#Attribute names in loom file, same as in standard kallisto|bustools --lamanno output
attribute_names=[('unspliced','spliced'),'gene_name','barcode']

loom_filepaths = ['/home/tchari/counts/cancer_dt/loom/all_cancer_dt.loom']*num_ks #All runs use the same loom file
print('loom_filepaths: ',loom_filepaths)

n_datasets = len(loom_filepaths)

loom_filepaths:  ['/home/tchari/counts/norman_crispr/loom/allcrispr.loom', '/home/tchari/counts/norman_crispr/loom/allcrispr.loom']


Filter cell barcodes

In [ ]:
#Filter for cell barcodes with reasonable UMI threshold, and filter for cells in selected conditions
cf = []
thr_lb = [10]*num_ks #4e3,Set UMI count filter for each K run

fig1,ax1 = plt.subplots(1,num_ks,figsize=(15,5))

bcs = bars
n_cells = S.shape[0]


for k in range(num_ks):
    filename = loom_filepaths[k]
    dataset_name = dataset_names[k]


    #Make knee plot with S+U counts
    monod.preprocess.knee_plot(S.T+U.T,ax1[k],viz=True,thr=thr_lb[k])
    cf_ = ((S.T+U.T).sum(0)>thr_lb[k])

    #Make filter for low count barcodes
    n_annot_bcs = len(meta['cell_barcode'])
    annot_bcs_in_loom = meta['cell_barcode'].isin(bcs).sum()
    annot_bcs_in_filt_loom = meta['cell_barcode'].isin(bcs[cf_]).sum()
    print(f'Dataset {dataset_name}. \n\t{len(bcs)} barcodes in loom, {cf_.sum()} pass filter. {n_annot_bcs} in annotations; of these, {annot_bcs_in_loom} in loom and {annot_bcs_in_filt_loom} in filtered loom.')

    #Select for barcodes in selected conditions, if applicable
    annot_bcs = meta[(meta['guide_identity'].isin(conds_only))]['cell_barcode'] #previously filt_ids with ctrls
    cf.append(np.isin(bcs,annot_bcs) & cf_)
    print(f'{len(annot_bcs)} cells in annotations. {np.isin(bcs,annot_bcs).sum()} in loom. {cf[-1].sum()} pass filter.')


    ax1[k].set_title(dataset_name)

!mkdir ./figs
fig_dir = './figs/'
fig_string = fig_dir + 'kneeplots_norman_meK.png'
fig1.tight_layout()
plt.savefig(fig_string,dpi=450)

#110,107 cells

## **Create Output Files and Run meK-Means**

In [906]:
import logging, sys
logging.basicConfig(stream=sys.stdout)
log = logging.getLogger()
log.setLevel(logging.INFO)
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

In [907]:
!mkdir ./fits

mkdir: cannot create directory ‘./fits’: File exists


In [ ]:
#Select to run inference on, same genes selected across all runs
dir_string,dataset_strings = monod.preprocess.construct_batch(loom_filepaths, \
                                             transcriptome_filepath, \
                                             dataset_names, \
                                             attribute_names=attribute_names,\
                                             batch_location='./fits',meta='meK_norman',batch_id=1,\
                                             datestring='230917', n_genes=len(all_gs_forinf),\
                                             exp_filter_threshold=None,cf=cf,
                                                             genes_to_fit=all_gs_forinf) #len(all_gs_forinf)

**Run meK-Means**

In [911]:
#Set bounds from physical parameters search space
phys_lb = [-2.0, -1.8, -1.8 ]
phys_ub = [4.2, 2.5, 2.5]


samp_lb = [-7.157894736842105, -1.525]
samp_ub = [-7.157894736842105, -1.525]

gridsize = [1,1]

In [912]:
#Set number of epochs
epochs = 10

In [ ]:
result_strings = []
for i in range(num_ks):

    #Define model with bursty transcription and Poisson molecule capture/sampling
    fitmodel = monod.cme_toolbox.CMEModel('Bursty','Poisson')

    #Set up mminference parameters
    inference_parameters = monod.mminference.InferenceParameters(phys_lb,phys_ub,samp_lb,samp_ub,gridsize,\
                dataset_strings[i],fitmodel,k=ks[i],epochs=epochs,use_lengths = True,
                gradient_params = {'max_iterations':5,'init_pattern':'moments','num_restarts':1})

    #Read in loom file with filtered barcodes
    search_data = monod.extract_data.extract_data(loom_filepaths[i], transcriptome_filepath, dataset_names[i],
                dataset_strings[i], dir_string, viz=False, dataset_attr_names=attribute_names,cf=cf[i])

    #Run inference(fit_all_grid_points()) and Save result file strings
    full_result_string = inference_parameters.fit_all_grid_points(30,search_data) 

    result_strings.append(full_result_string)
    
    